# Arabic Sentiment Analysis in tweets using Naive Bayes Machine learning Algorithm and unigram features

In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import nltk
from nltk import NaiveBayesClassifier
from nltk.metrics.scores import f_measure, precision, recall
import collections


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
for filename in os.listdir("C:\\Users\\Humna Shafqat\\Desktop\\python\\sentimant_analysis\\input"):
    print(filename)

# Any results you write to the current directory are saved as output.

3labels
Arabic_tweets_negative_20190413.tsv
Arabic_tweets_positive_20190413.tsv
test_Arabic_tweets_negative_20190413.tsv
test_Arabic_tweets_positive_20190413.tsv
train_Arabic_tweets_negative_20190413.tsv
train_Arabic_tweets_positive_20190413.tsv


# define functions 

In [77]:
import re
from itertools import islice

def load_tsv(data_file, n):
    data_features = list()
    data = list()
    infile = open(data_file, encoding='utf-8')
    for line in infile:
        if not line.strip():
            continue
        label, text = line.split('\t')
        text_features = process_text(text, n)
        if text_features:
            data_features += text_features
            data.append((text_features, label))
    return data, data_features

def process_text(text, n=1,
                 remove_vowel_marks=False,
                 remove_repeated_chars=False,
                 ):
    clean_text = text
    if remove_vowel_marks:
        clean_text = remove_diacritics(clean_text)
    if remove_repeated_chars:
        clean_text = remove_repeating_char(clean_text)

    if n == 1:
        return clean_text.split()
    else:
        tokens = clean_text.split()
        grams = tokens
        for i in range(2, n + 1):
            grams += [  ' '.join(g) for g in list(window(tokens, i))  ]
        return grams



def window(words_seq, n):
    """Returns a sliding window (of width n) over data from the iterable"""
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(words_seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result


def remove_repeating_char(text):
    # return re.sub(r'(.)\1+', r'\1', text)     # keep only 1 repeat
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeat

def document_features(document, corpus_features):
    document_words = set(document)
    features = {}
    for word in corpus_features:
        features['has({})'.format(word)] = (word in document_words)
    return features

# Load corpus

In [78]:
pos_train_file = "input\\train_Arabic_tweets_positive_20190413.tsv"

neg_train_file = "input\\train_Arabic_tweets_negative_20190413.tsv"

pos_test_file = "input\\test_Arabic_tweets_positive_20190413.tsv"

neg_test_file = "input\\test_Arabic_tweets_negative_20190413.tsv"
my_test=r"C:\Users\Humna Shafqat\Desktop\python\sentimant_analysis\ten.tsv"
print('data files')
print('train file (pos)', pos_train_file)
print('train file (neg)', neg_train_file)
print('test file (pos)', pos_test_file)
print('test file (neg)', neg_test_file)
print("my test file",my_test)

data files
train file (pos) input\train_Arabic_tweets_positive_20190413.tsv
train file (neg) input\train_Arabic_tweets_negative_20190413.tsv
test file (pos) input\test_Arabic_tweets_positive_20190413.tsv
test file (neg) input\test_Arabic_tweets_negative_20190413.tsv
my test file C:\Users\Humna Shafqat\Desktop\python\sentimant_analysis\ten.tsv


# Parameters (ngrams)

In [79]:
print('parameters')
n = 1
print('n grams:', n)

parameters
n grams: 1


# loading train data .... 

In [80]:
print('loading train data ....')
pos_train_data, pos_train_feat = load_tsv(pos_train_file, n)
neg_train_data, neg_train_feat = load_tsv(neg_train_file, n)
print('loading test data ....')
pos_test_data, pos_test_feat = load_tsv(pos_test_file, n)
neg_test_data, neg_test_feat = load_tsv(neg_test_file, n)
n,p=load_tsv(my_test,n)

loading train data ....
loading test data ....


In [81]:
pos_train_data[900]

(['الواحد',
  'الفتره',
  'دى',
  'أعوذ',
  'بالله',
  'يعنى',
  'لا',
  'قدر',
  'الله',
  'بيفكر',
  'فى',
  'الارتباط',
  '😂',
  '—',
  'وحسبى',
  'الله',
  'ونعم',
  'الوكيل',
  '🤣🤣'],
 'pos')

In [82]:
print(str(n)+"                          "+str(p))

[(['"@stcpay_ksa', 'شركة', 'سخيفه', 'ارباحها', 'بالمليارات', 'وواجبها', 'تجاه', 'المجتمع', 'صفر', '!'], 'neg'), (['ماهو', 'رأيكم', 'تقيمون', 'مسابقه', 'عن', 'اجمل', 'صوت', 'لحفظة', 'القرآن', 'الكريم'], 'pos'), (['يومك', 'سعيد'], 'pos'), (['🎨', 'بيع', 'حسابات', 'انستقرام', '🇸🇦', 'للبيع', '\u2066♥️\u2069', '👌مناسبة', 'لل🎼', 'متاجر', '🎼', 'حسابات', 'شخصية', '..', 'حساب', 'للبيع', '👌', '\u2066\u2066♥️\u2069\u20695', 'آلاف', 'متابع', '⚫7', 'آلاف', 'متابع', '\u2066♥️\u206911', 'ألف', 'متابع', '\u2066📉\u2069\u2069', 'نقبل', 'تحويل', 'بنكي', 'ع', 'الاهلي', 'والراجحي', 'وشحن', 'STC', '\u2066', 'للتواصل', 'خاص', 'فقط🎨', '#ماذا_يغضب_البنات', 'https://t.co/f5mMpXaR7d'], 'pos'), (['RT', '@RagadAlsaqqa', ':', 'اشياء', 'لا', 'يراها', 'سوى', 'مستخدمو', '@stc_ksa', '😤', 'https://t.co/3M9U6TreT3'], 'neg'), (['بعد', '١٢', 'سنة', 'مع', '#موبايلي', 'في', 'باقة', 'راقي', 'قررت', 'احول', 'الى', '#stc', 'بسبب', 'سوء', 'تغطية', 'شبكة', '#موبايلي', 'للاسف', '💔', 'احب', 'موبايلي', 'بس', 'معد', 'اقدر', 'اتحمل', '

# Training data information

In [83]:
print('train data info')
train_data = pos_train_data + neg_train_data
print('train data size', len(train_data))
print('# of positive', len(pos_train_data))
print('# of negative', len(neg_train_data))

train data info
train data size 47000
# of positive 23879
# of negative 23121


In [84]:
train_data[0]


(['نحن',
  'الذين',
  'يتحول',
  'كل',
  'ما',
  'نود',
  'أن',
  'نقوله',
  'إلى',
  'دعاء',
  'لله،',
  'لا',
  'تبحثوا',
  'فينا',
  'عن',
  'قوة،',
  'إننا',
  'مكسورون،',
  'القوة',
  'التي…'],
 'pos')

# Sample training data 

In [85]:
import random
sample_size = 1
print('{} random tweets .... '.format(sample_size))
for s in random.sample(train_data, sample_size):
    print(s)

1 random tweets .... 
(['شيء', 'مؤسف', 'فعلا', '💔'], 'neg')


# Test data info

In [86]:
print('test data info')
test_data = pos_test_data + neg_test_data
print('test data size', len(train_data))
print('# of positive', len(pos_test_data))
print('# of negative', len(neg_test_data))

test data info
test data size 47000
# of positive 5970
# of negative 5781


features are all the word in the file with repeation


# merging all features ...

In [87]:
print('merging all features ... ')
all_features = pos_train_feat + neg_train_feat + \
               pos_test_feat + pos_test_feat
print('len(all_features):', len(all_features))

merging all features ... 
len(all_features): 770508


# Sample features 

In [88]:
print('{} sample features ...'.format(sample_size))
print(random.sample(all_features, sample_size))

1 sample features ...
['الحلم']


# compute frequencies

In [89]:
all_features_count = {}
for w in all_features:
    all_features_count[w] = all_features_count.get(w, 0) + 1

# Sample Frequency

In [90]:
print('sample frequencies')
print(random.sample(list(all_features_count.items()), 30))
word = 'في'
print('freq of word {} is {}'.format(word, all_features_count.get(word, 0)))
word = 'فى'
print('freq of word {} is {}'.format(word, all_features_count.get(word, 0)))
word = 'من'
print('freq of word {} is {}'.format(word, all_features_count.get(word, 0)))

sample frequencies
[('التل', 1), ('زمنٰ', 2), ('طبعا😌', 1), ('تقييما', 1), ('بعدها', 45), ('للكرة', 1), ('معاوية', 1), ('بساطة', 33), ('التريلر', 1), ('مدركين', 2), ('كتوما', 1), ('بسيارة', 2), ('شخصآ', 3), (':⤵', 2), ('مالج', 1), ('ضوء', 24), ('داعسك', 2), ('حادث', 9), ('ل/', 1), ('تعلمهم', 4), ('هز', 12), ('#تووق', 1), ('أه…', 1), ('لمدارس', 1), ('سجدت..', 1), ('سلوتي', 1), ('ونه', 1), ('التفاعل)بدل', 2), ('~♡', 1), ('#ولي_العهد_الأمير_محمد_بن_سلمان', 2)]
freq of word في is 9550
freq of word فى is 220
freq of word من is 12655


# Compute Threshold

In [91]:
print('size of training data:',  len(train_data))
min_df = int(0.001 * len(train_data))
max_df = int(0.98 * len(train_data))
print('min document frequency:', min_df)
print('max document frequency:', max_df)

size of training data: 47000
min document frequency: 47
max document frequency: 46060


# Selecting Features 

In [92]:
# remove features that have frequency below/above the threshold
my_features = set([word for word, freq in all_features_count.items() if  max_df > freq > min_df ])
print(len(my_features), 'are kept out of', len(all_features))

1961 are kept out of 770508


# Sample of selected features 

In [93]:
print('{} sample of selected features:'.format(sample_size))
print(random.sample(list(my_features), sample_size))

1 sample of selected features:
['عين']


# generating features for training documents ...

In [ ]:
feature_sets = [(document_features(d, my_features), c) for (d, c) in train_data]


In [23]:
type(feature_sets)

list

In [24]:
len(feature_sets)

47000

In [25]:
feature_sets[666]

({'has(النور)': False,
  'has(😌)': False,
  'has(..)': False,
  'has(الليل)': False,
  'has(نص)': False,
  'has(بني)': False,
  'has(ساعات)': False,
  'has(صبااح)': False,
  'has(مالي)': False,
  'has(ولا)': False,
  'has(ملاحظة)': False,
  'has(جديد)': False,
  'has(الله)': False,
  'has(الورد)': False,
  'has(ورد)': False,
  'has(المهم)': False,
  'has(كله)': False,
  'has(ابو)': False,
  'has(🤩)': False,
  'has(سعودي)': False,
  'has(معاكم)': False,
  'has(آللهم)': False,
  'has(ثلاث)': False,
  'has(دا)': False,
  'has(#النصر)': False,
  'has(الأمل)': False,
  'has(مواقف)': False,
  'has(وبين)': False,
  'has(صفراء)': False,
  'has(اسمع)': False,
  'has(الصحة)': False,
  'has(بكره)': False,
  'has(وأن)': False,
  'has(كلهم)': False,
  'has(معاه)': False,
  'has(صعب)': False,
  'has(عنده)': False,
  'has(الرضا)': False,
  'has(يسعدك)': False,
  'has(أليمة)': False,
  'has(قل)': False,
  'has(🎀)': False,
  'has(ماني)': False,
  'has(ص…)': False,
  'has(يمكن)': False,
  'has(حلوه)': F

# training ...

In [26]:
classifier = nltk.NaiveBayesClassifier.train(feature_sets)
print('training is done')

training is done


# Most informative features 

In [27]:
classifier.show_most_informative_features(10)

Most Informative Features
               has(موثق) = True              pos : neg    =    238.5 : 1.0
                  has(😭) = True              neg : pos    =    202.0 : 1.0
                  has(😢) = True              neg : pos    =    171.3 : 1.0
            has(المسيار) = True              pos : neg    =    170.1 : 1.0
              has(وصلوا) = True              pos : neg    =    166.9 : 1.0
                  has(😳) = True              neg : pos    =    164.2 : 1.0
             has(الشروط) = True              pos : neg    =    151.4 : 1.0
              has(وتابع) = True              pos : neg    =    143.9 : 1.0
               has(ببكي) = True              neg : pos    =    143.6 : 1.0
                  has(🥀) = True              neg : pos    =    132.4 : 1.0


# generating features for test documents ...

In [28]:
test_features = [(document_features(d, my_features), c) for (d, c) in test_data]

In [29]:
lil_test=[(document_features(d, my_features), c) for (d, c) in n]

In [58]:
lil_test[1]

({'has(فإن)': False,
  'has(طالب)': False,
  'has(ألف)': False,
  'has(عمان)': False,
  'has(عطا)': False,
  'has(!!)': False,
  'has(💖)': False,
  'has(العربية)': False,
  'has(أبو)': False,
  'has(رصيد)': False,
  'has(تاني)': False,
  'has(تعالى)': False,
  'has(%)': False,
  'has(شديدة)': False,
  'has(😖)': False,
  'has(ودي)': False,
  'has(نظام)': False,
  'has(حكم)': False,
  'has(خيرا)': False,
  'has(🥀)': False,
  'has(يابو)': False,
  'has(والاحتفال)': False,
  'has(أسألك)': False,
  'has(اجعلنا)': False,
  'has(آدم)': False,
  'has(توجيه)': False,
  'has(تتخاذل)': False,
  'has(وربي)': False,
  'has(هندي)': False,
  'has(ㅤ)': False,
  'has(عاد)': False,
  'has(الإتحاد)': False,
  'has(بنات)': False,
  'has(السوداني)': False,
  'has(الرجل)': False,
  'has(🎵)': False,
  'has(فض)': False,
  'has(يكفي)': False,
  'has(الصوت)': False,
  'has(مواقف)': False,
  'has(👌)': False,
  'has(بشكل)': False,
  'has(هالشي)': False,
  'has(❥)': False,
  'has(ماشاءالله)': False,
  'has(الحين)'

In [30]:
len(test_features)

11751

# classify test instances 

In [31]:
ref_sets = collections.defaultdict(set)
test_sets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_features):
    ref_sets[label].add(i)
    observed = classifier.classify(feats)
    test_sets[observed].add(i)

In [141]:
test_features[0]

({'has(النور)': False,
  'has(😌)': False,
  'has(..)': False,
  'has(الليل)': False,
  'has(نص)': False,
  'has(بني)': False,
  'has(ساعات)': False,
  'has(صبااح)': False,
  'has(مالي)': False,
  'has(ولا)': False,
  'has(ملاحظة)': False,
  'has(جديد)': False,
  'has(الله)': False,
  'has(الورد)': False,
  'has(ورد)': False,
  'has(المهم)': False,
  'has(كله)': False,
  'has(ابو)': False,
  'has(🤩)': False,
  'has(سعودي)': False,
  'has(معاكم)': False,
  'has(آللهم)': False,
  'has(ثلاث)': False,
  'has(دا)': False,
  'has(#النصر)': False,
  'has(الأمل)': False,
  'has(مواقف)': False,
  'has(وبين)': False,
  'has(صفراء)': False,
  'has(اسمع)': False,
  'has(الصحة)': False,
  'has(بكره)': False,
  'has(وأن)': False,
  'has(كلهم)': False,
  'has(معاه)': False,
  'has(صعب)': False,
  'has(عنده)': False,
  'has(الرضا)': False,
  'has(يسعدك)': False,
  'has(أليمة)': False,
  'has(قل)': False,
  'has(🎀)': False,
  'has(ماني)': False,
  'has(ص…)': False,
  'has(يمكن)': False,
  'has(حلوه)': F

# Results 

In [32]:
print('accuracy: ', nltk.classify.accuracy(classifier, test_features))
print('pos precision: ', precision(ref_sets['pos'], test_sets['pos']))
print('pos recall:', recall(ref_sets['pos'], test_sets['pos']))
print('neg precision: ', precision(ref_sets['neg'], test_sets['neg']))
print('neg recall:', recall(ref_sets['neg'], test_sets['neg']))
print('positive f-score:', f_measure(ref_sets['pos'], test_sets['pos']))
print('negative f-score:', f_measure(ref_sets['neg'], test_sets['neg']))

accuracy:  0.8913283975831844
pos precision:  0.9198425478618716
pos recall: 0.8611390284757119
neg precision:  0.8654657578708211
neg recall: 0.9225047569624633
positive f-score: 0.8895233151656718
negative f-score: 0.8930754416813196


In [38]:
# infile = open(path, encoding='utf-8')
# for line in infile:
#     print(line)

In [39]:
from nltk.tokenize import word_tokenize

In [33]:
for i, (feats, label) in enumerate(lil_test):
    ref_sets[label].add(i)
    observed = classifier.classify(feats)
    test_sets[observed].add(i)

In [35]:
print('accuracy: ', nltk.classify.accuracy(classifier,lil_test))

accuracy:  0.6666666666666666


In [89]:
nltk.classify.accuracy(classifier,lil_test)

0.6666666666666666

In [98]:
#a=classifier.classify(lil_test)

In [39]:
t= pd.read_csv('tweet_text.csv', delimiter = '\t') 

In [101]:
import pickle

In [102]:
# pickle_out=open("classifier.pickle","wb")
# pickle.dump(classifier,pickle_out)
# pickle_out.close()

In [103]:
# pickle_out=open("my_features.pickle","wb")
# pickle.dump(my_features,pickle_out)
# pickle_out.close()

In [99]:
from nltk.tokenize import word_tokenize

In [109]:
my_data = pd.read_csv('preprocessed.tsv', delimiter = '\t') 
my_data.head()

,@FarohaM كل ابراج stc
0,RT @ins660 : 🎨 بيع حسابات انستقرام 🇸🇦 للبيع ⁦♥...
1,RT @m_madridi : مواجهات نادي القادسية في شهر ف...
2,RT @3rbawei93 : غيابات العربي غداً أمام الساحل...
3,@M3r5al مساء الخير محمد عشان نقوم معاك بالواج...
4,غيابات العربي غداً أمام الساحل في دوري STC الم...


In [1]:
my_data=my_data.rename(columns={"@FarohaM كل ابراج stc":"tweets"})
my_data.head()

NameError: name 'my_data' is not defined

In [121]:
for i in range(0,10): 
    # column : "Review", row ith ()
    tweets =my_data['tweets'][i] 
    tw=word_tokenize(tweets)
    tf = [{(document_features(d, my_features)) for d in tw}]
    print(tf)
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[{'has(النور)': False, 'has(😌)': False, 'has(..)': False, 'has(الليل)': False, 'has(نص)': False, 'has(بني)': False, 'has(ساعات)': False, 'has(صبااح)': False, 'has(مالي)': False, 'has(ولا)': False, 'has(ملاحظة)': False, 'has(جديد)': False, 'has(الله)': False, 'has(الورد)': False, 'has(ورد)': False, 'has(المهم)': False, 'has(كله)': False, 'has(ابو)': False, 'has(🤩)': False, 'has(سعودي)': False, 'has(معاكم)': False, 'has(آللهم)': False, 'has(ثلاث)': False, 'has(دا)': False, 'has(#النصر)': False, 'has(الأمل)': False, 'has(مواقف)': False, 'has(وبين)': False, 'has(صفراء)': False, 'has(اسمع)': False, 'has(الصحة)': False, 'has(بكره)': False, 'has(وأن)': False, 'has(كلهم)': False, 'has(معاه)': False, 'has(صعب)': False, 'has(عنده)': False, 'has(الرضا)': False, 'has(يسعدك)': False, 'has(أليمة)': False, 'has(قل)': False, 'has(🎀)': False, 'has(ماني)': False, 'has(ص…)': False, 'has(يمكن)': False, 'has(حلوه)': False, 'has(كتب)': False, 'has(الوقت)': False, 'has(التغريدة)': False, 'has(واسمع)': False,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [138]:
tweets =my_data['tweets'][1] 
tw=word_tokenize(tweets)
tf = [(document_features(d, my_features)) for d in tw]
print(tf[1])

{'has(النور)': False, 'has(😌)': False, 'has(..)': False, 'has(الليل)': False, 'has(نص)': False, 'has(بني)': False, 'has(ساعات)': False, 'has(صبااح)': False, 'has(مالي)': False, 'has(ولا)': False, 'has(ملاحظة)': False, 'has(جديد)': False, 'has(الله)': False, 'has(الورد)': False, 'has(ورد)': False, 'has(المهم)': False, 'has(كله)': False, 'has(ابو)': False, 'has(🤩)': False, 'has(سعودي)': False, 'has(معاكم)': False, 'has(آللهم)': False, 'has(ثلاث)': False, 'has(دا)': False, 'has(#النصر)': False, 'has(الأمل)': False, 'has(مواقف)': False, 'has(وبين)': False, 'has(صفراء)': False, 'has(اسمع)': False, 'has(الصحة)': False, 'has(بكره)': False, 'has(وأن)': False, 'has(كلهم)': False, 'has(معاه)': False, 'has(صعب)': False, 'has(عنده)': False, 'has(الرضا)': False, 'has(يسعدك)': False, 'has(أليمة)': False, 'has(قل)': False, 'has(🎀)': False, 'has(ماني)': False, 'has(ص…)': False, 'has(يمكن)': False, 'has(حلوه)': False, 'has(كتب)': False, 'has(الوقت)': False, 'has(التغريدة)': False, 'has(واسمع)': False, 

In [140]:
print(tf[0])

{'has(النور)': False, 'has(😌)': False, 'has(..)': False, 'has(الليل)': False, 'has(نص)': False, 'has(بني)': False, 'has(ساعات)': False, 'has(صبااح)': False, 'has(مالي)': False, 'has(ولا)': False, 'has(ملاحظة)': False, 'has(جديد)': False, 'has(الله)': False, 'has(الورد)': False, 'has(ورد)': False, 'has(المهم)': False, 'has(كله)': False, 'has(ابو)': False, 'has(🤩)': False, 'has(سعودي)': False, 'has(معاكم)': False, 'has(آللهم)': False, 'has(ثلاث)': False, 'has(دا)': False, 'has(#النصر)': False, 'has(الأمل)': False, 'has(مواقف)': False, 'has(وبين)': False, 'has(صفراء)': False, 'has(اسمع)': False, 'has(الصحة)': False, 'has(بكره)': False, 'has(وأن)': False, 'has(كلهم)': False, 'has(معاه)': False, 'has(صعب)': False, 'has(عنده)': False, 'has(الرضا)': False, 'has(يسعدك)': False, 'has(أليمة)': False, 'has(قل)': False, 'has(🎀)': False, 'has(ماني)': False, 'has(ص…)': False, 'has(يمكن)': False, 'has(حلوه)': False, 'has(كتب)': False, 'has(الوقت)': False, 'has(التغريدة)': False, 'has(واسمع)': False, 

In [128]:
tweets =my_data['tweets'][1]
tweets

'RT @m_madridi : مواجهات نادي القادسية في شهر فبراير : الكويت { STC } 6 فبراير ✈️ الرفاع البحريني { AFC } 11فبراير ✈️ اليرموك { STC } 15 فبراير 🏠 ظفار العماني{ AFC } 25فبراير 🏠#القادسية'

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
classifier=pickle.load(open('classifier.pickle','rb'))

In [4]:
my_features=pickle.load(open('my_features.pickle','rb'))

In [5]:
from nltk.tokenize import word_tokenize

In [6]:
my_data = pd.read_csv('preprocessed.tsv', delimiter = '\t') 
my_data.head()

,@FarohaM كل ابراج stc
0,RT @ins660 : 🎨 بيع حسابات انستقرام 🇸🇦 للبيع ⁦♥...
1,RT @m_madridi : مواجهات نادي القادسية في شهر ف...
2,RT @3rbawei93 : غيابات العربي غداً أمام الساحل...
3,@M3r5al مساء الخير محمد عشان نقوم معاك بالواج...
4,غيابات العربي غداً أمام الساحل في دوري STC الم...


In [7]:
my_data=my_data.rename(columns={"@FarohaM كل ابراج stc":"tweets"})
my_data.head()

,tweets
0,RT @ins660 : 🎨 بيع حسابات انستقرام 🇸🇦 للبيع ⁦♥...
1,RT @m_madridi : مواجهات نادي القادسية في شهر ف...
2,RT @3rbawei93 : غيابات العربي غداً أمام الساحل...
3,@M3r5al مساء الخير محمد عشان نقوم معاك بالواج...
4,غيابات العربي غداً أمام الساحل في دوري STC الم...


In [8]:
len(my_features)

1961

In [9]:
my_feature_list=list(my_features)

In [10]:
my_feature_list[1960]


'ليش'

In [56]:
tweet="😳 🥀 😭"
tw=word_tokenize(tweet)

In [57]:
len(tw)

3

In [58]:
tw[0]

'😳'

In [59]:
my_dict = {"My_Features":my_feature_list}

In [60]:
pred_DataFrame=pd.DataFrame(my_dict)

In [61]:
pred_DataFrame.head()

,My_Features
0,لقاء
1,واسمع
2,حسن
3,لهم
4,شلون


In [62]:
len(pred_DataFrame)

1961

In [63]:
list2=[]
list2=tw
list2

['😳', '🥀', '😭']

In [64]:
array1=np.array(my_feature_list)

In [65]:
df_final=pd.DataFrame(columns=array1)
df_final.head()

,لقاء,واسمع,حسن,لهم,شلون,🔴,متمسك,…,تقول,❤️,...,الحياة,الكثير,الشباب,انو,القادمة,كلمتين,🌼,آخر,نبي,ليش


In [66]:
for index in range(len(array1)):
    if df_final.columns[index] in list2:
        df_final.loc[0,df_final.columns[index]]=1
    else:
        df_final.loc[0,df_final.columns[index]]=0

In [67]:
df_final.head()

,لقاء,واسمع,حسن,لهم,شلون,🔴,متمسك,…,تقول,❤️,...,الحياة,الكثير,الشباب,انو,القادمة,كلمتين,🌼,آخر,نبي,ليش
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
for value in range(len(df_final.columns)):
    if df_final.loc[0,df_final.columns[value]]==1:
        print(df_final.loc[0,df_final.columns[value]])
        print(df_final.columns[value],value)
    else:
        pass

1
😭 328
1
😳 781
1
🥀 1679


In [69]:
for value in range(len(df_final.columns)):
    if df_final.loc[0,df_final.columns[value]]==1:
        print(df_final.loc[0,df_final.columns[value]])
        print(df_final.columns[value],value)
        df_final.loc[0,df_final.columns[value]]=True
    else:
        df_final.loc[0,df_final.columns[value]]=False

1
😭 328
1
😳 781
1
🥀 1679


In [70]:
df_final.head()

,لقاء,واسمع,حسن,لهم,شلون,🔴,متمسك,…,تقول,❤️,...,الحياة,الكثير,الشباب,انو,القادمة,كلمتين,🌼,آخر,نبي,ليش
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [71]:
result_df=df_final.transpose()
result_df.head()

,0
لقاء,False
واسمع,False
حسن,False
لهم,False
شلون,False


In [72]:
my_dict=result_df.to_dict()

In [73]:
my_dict

{0: {'لقاء': False,
  'واسمع': False,
  'حسن': False,
  'لهم': False,
  'شلون': False,
  '🔴': False,
  'متمسك': False,
  '…': False,
  'تقول': False,
  '❤️': False,
  'السورة': False,
  '⇣': False,
  'فض': False,
  'استغفر': False,
  'نشبة': False,
  'عندنا': False,
  'تافه': False,
  'بعدين': False,
  '📍': False,
  'تفتح': False,
  'انسى': False,
  'الهلال': False,
  'اليوم': False,
  'ان': False,
  'الماء': False,
  'سبحانه': False,
  '😆': False,
  'يدخل': False,
  'شروط': False,
  '💞': False,
  'نوبى': False,
  'عمانية': False,
  'حصل': False,
  '😉': False,
  'احبك': False,
  'الليله': False,
  'وسلموا': False,
  'زمان': False,
  'يموت': False,
  'كلنا': False,
  'لكننا': False,
  'ياما': False,
  'أخذ': False,
  'نجم': False,
  'وزنك': False,
  'قدر': False,
  'مكان': False,
  'النفس': False,
  'السلام': False,
  'بني': False,
  'يأتي': False,
  'الوحيد': False,
  'مريض': False,
  'ا…': False,
  'دقيقة': False,
  'فرحا': False,
  'لازم': False,
  'مبلغ': False,
  'بين': False,
  '-

In [74]:
print(classifier.classify(my_dict))

pos


In [53]:
#final_dict=my_dict[0]

In [54]:
#final_dict

In [200]:
# fn=[]
# fn.append(final_dict)

In [201]:
#print(fn)

In [195]:
test_data = [{"❤️": False, "الكويت": True, "إنتعشتك": False}]

In [203]:
 t =my_data['tweets'][1]

In [204]:
print(t)

RT @m_madridi : مواجهات نادي القادسية في شهر فبراير : الكويت { STC } 6 فبراير ✈️ الرفاع البحريني { AFC } 11فبراير ✈️ اليرموك { STC } 15 فبراير 🏠 ظفار العماني{ AFC } 25فبراير 🏠#القادسية


In [75]:
print(classifier.classify(my_dict))

pos


In [192]:
# from nltk.classify import SklearnClassifier
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.svm import SVC
# # train_data = [({"a": 4, "b": 1, "c": 0}, "ham"),
# # ...               ({"a": 5, "b": 2, "c": 1}, "ham"),
# # ...               ({"a": 0, "b": 3, "c": 4}, "spam"),
# # ...               ({"a": 5, "b": 1, "c": 1}, "ham"),
# # ...               ({"a": 1, "b": 4, "c": 3}, "spam")]
# classif = SklearnClassifier(BernoulliNB()).train(feature_sets)
# test_data = [{"a": 3, "b": 2, "c": 1},
#              {"a": 0, "b": 3, "c": 7}]
# a=classif.classify(fn)

# # >>> classif = SklearnClassifier(SVC(), sparse=False).train(train_data)
# # >>> classif.classify_many(test_data)


KeyboardInterrupt: 